In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
and customer_location_hex_8 in ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a05fffff', '8860a24a0dfffff', '8860a24a21fffff',
  '8860a24a25fffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a45fffff', '8860a24a4dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff',
   '8860a24b09fffff', '8860a24b0bfffff', '8860a24b19fffff', '8860a24b1dfffff', '8860a24b43fffff', '8860a24b49fffff', '8860a24b4bfffff', '8860a24b53fffff',
    '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25981fffff', '8860a25983fffff', '8860a25985fffff',
     '8860a25987fffff', '8860a2598bfffff', '8860a25991fffff', '8860a25993fffff', '8860a25999fffff', '8860a2599bfffff', '8860a2599dfffff', '8860a259a1fffff',
      '8860a259a3fffff', '8860a259a7fffff', '8860a259a9fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff', '8860a259d7fffff')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Query data for getting the gross number of captains
q_captain = """
select pickup_location_hex_8, yyyymmdd, order_type, hour, avg(gross_captains) as gross_captains from
(select pickup_location_hex_8, yyyymmdd, order_type, hour, min_1, sec_window, count(distinct captain_id) as gross_captains from 
(select 
        order_id, updated_epoch, yyyymmdd, pickup_location_hex_8, captain_location_hex_8, drop_location_hex_8 as hex_id,
        city_name, order_type, captain_id, event_type, order_status, hhmmss as time, substr(hhmmss,1,2) as hour, substr(hhmmss,3,4) as min_1,
        (case when cast(substr(hhmmss,5,6) as real) >= 0 and cast(substr(hhmmss,5,6) as real) <= 20 then '0-20'
        when cast(substr(hhmmss,5,6) as real) >= 20 and cast(substr(hhmmss,5,6) as real) <= 40 then '20-40'
        when cast(substr(hhmmss,5,6) as real) >= 40 and cast(substr(hhmmss,5,6) as real) <= 60 then '40-60' end) as sec_window,
        cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count,
        array_agg(event_type) over (partition by order_id) as event_type_agg, updated_hhmmss
        
from 
        orders.order_logs_immutable
where 
        yyyymmdd >= '20220919'
        and yyyymmdd <= '20221016'
        and pickup_location_hex_8 IN ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a05fffff', '8860a24a0dfffff', '8860a24a21fffff',
  '8860a24a25fffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a45fffff', '8860a24a4dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff',
   '8860a24b09fffff', '8860a24b0bfffff', '8860a24b19fffff', '8860a24b1dfffff', '8860a24b43fffff', '8860a24b49fffff', '8860a24b4bfffff', '8860a24b53fffff',
    '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25981fffff', '8860a25983fffff', '8860a25985fffff',
     '8860a25987fffff', '8860a2598bfffff', '8860a25991fffff', '8860a25993fffff', '8860a25999fffff', '8860a2599bfffff', '8860a2599dfffff', '8860a259a1fffff',
      '8860a259a3fffff', '8860a259a7fffff', '8860a259a9fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff', '8860a259d7fffff')
       and order_type IN ('app'))
group by 1, 2, 3, 4, 5, 6)
group by 1, 2, 3, 4
"""

df_captain_copy = pd.read_sql(q_captain, connection)

In [5]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta
0,634735081b4dce7515f90f0e,2.926,Hyderabad,8860a259abfffff,17.454550,78.399109,031336,03,0300,Link,03:00,4,20221013,customerCancelled,Change of plans,7.0
1,6347358aa51bb634fb9202ff,NaN,Hyderabad,8860a2599dfffff,17.432936,78.387047,031546,03,0315,Link,03:00,4,20221013,customerCancelled,order cancelled before rider accepted,2.0
2,6347382719caea40e13ad9bc,NaN,Hyderabad,8860a259b9fffff,17.441704,78.384491,032655,03,0315,Link,03:00,4,20221013,expired,None,7.0
3,63473aa01b4dce7515f910a7,NaN,Hyderabad,8860a24b59fffff,17.457399,78.373138,033728,03,0330,Link,03:59,4,20221013,expired,None,3.0
4,63473c4c19caea40e13adade,NaN,Hyderabad,8860a259b3fffff,17.447933,78.374626,034436,03,0330,Link,03:59,4,20221013,customerCancelled,order cancelled before rider accepted,7.0


In [6]:
#View the dataset
df_captain = df_captain_copy.copy()
df_captain.head()

,pickup_location_hex_8,yyyymmdd,order_type,hour,gross_captains
0,8860a259a3fffff,20221016,app,17,2.000000
1,8860a25995fffff,20220930,app,18,3.254098
2,8860a259bbfffff,20220921,app,16,2.165414
3,8860a24b51fffff,20220923,app,19,2.421875
4,8860a259b1fffff,20220925,app,10,2.274510


In [20]:
#Load datasets including hexes from level_1 and level_2
df_level_1 = pd.read_csv('hyderabad_location_data_level1_demand_zone2.csv')

level1_hexes = list(df_level_1['customer_location_hex_8'].unique())

#Filter only on the hexes needed
df_filter_level1 = df[df['customer_location_hex_8'].isin(level1_hexes)]
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].apply(lambda x: x[6:])

In [21]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA'))

In [22]:
#Zone-level-metrics, calculated on per hour basis

hourly_metrics = []
date_metrics = {}
for date in df_filter_level1['date'].unique().tolist():
    df_date = df_filter_level1[df_filter_level1['date'] == date]
    for hour in list(df_date['hour'].unique()):
        df_hourly_level1 = df_date[df_date['hour'] == hour]

        #OCARA
        df_cancel_level1 = df_hourly_level1[df_hourly_level1['order_status'] == 'customerCancelled']

        ocara_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'OCARA'].shape[0]/df_hourly_level1.shape[0]), 4)
        #COBRA
        cobra_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'COBRA'].shape[0]/df_hourly_level1.shape[0]), 4)

        #COBRM
        cobrm_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'COBRM'].shape[0]/df_hourly_level1.shape[0]), 4)

        #G2N
        g2n_zone_level1 = round((df_hourly_level1[df_hourly_level1['order_status'] == 'dropped'].shape[0]/df_hourly_level1.shape[0]), 4)

        #ETA
        eta_zone_level1 = round(df_hourly_level1['eta'].mean(), 2)

        #FM
        dropped_rides_level_1 = df_hourly_level1[df_hourly_level1['order_status'] == 'dropped']
        fm_zone_level1 = round(dropped_rides_level_1['accept_to_pickup_distance'].mean(), 2)

        #Create list of different metrics
        level1_metrics = [date, hour, ocara_zone_level1, cobra_zone_level1, cobrm_zone_level1, g2n_zone_level1, eta_zone_level1, fm_zone_level1]
        hourly_metrics.append(level1_metrics)
metrics_df = pd.DataFrame(hourly_metrics)
metrics_df.columns = ['date', 'hour', 'OCARA', 'COBRA', 'COBRM', 'G2N', 'ETA', 'FM']
metrics_df = metrics_df.sort_values(by = ['date', 'hour'], ascending=True)
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM
355,01,00,0.3889,0.0617,0.0000,0.5494,2.96,0.84
337,01,01,0.4545,0.0505,0.0000,0.4949,2.81,0.71
348,01,02,0.4744,0.0769,0.0000,0.4487,2.65,0.65
351,01,03,0.3971,0.1176,0.0000,0.4853,3.16,0.98
336,01,04,0.3203,0.1046,0.0000,0.5686,3.92,1.18
...,...,...,...,...,...,...,...,...
12,30,19,0.2753,0.0531,0.0173,0.6432,3.18,0.65
20,30,20,0.2423,0.0543,0.0220,0.6725,3.18,0.67
14,30,21,0.2374,0.0747,0.0154,0.6549,3.33,0.61
18,30,22,0.2667,0.0578,0.0000,0.6667,3.16,0.57


In [23]:
#Get the number of rides by order status
rides_by_order = pd.pivot_table(df_filter_level1, index = ['date', 'hour'], columns = 'order_status', values = 'count', aggfunc = 'sum',
 fill_value=0).reset_index().rename_axis(None, axis=1)
rides_by_order['total'] = rides_by_order[['aborted', 'customerCancelled', 'dropped', 'expired']].sum(axis = 1)
rides_by_order['Expiry %'] = round(rides_by_order['expired']/rides_by_order['total'], 4)

#Merge with metrics
metrics_df = metrics_df.merge(rides_by_order[['date', 'hour', 'Expiry %', 'total']], on = ['date', 'hour'], how = 'left')
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total
0,01,00,0.3889,0.0617,0.0000,0.5494,2.96,0.84,0.0000,162
1,01,01,0.4545,0.0505,0.0000,0.4949,2.81,0.71,0.0000,99
2,01,02,0.4744,0.0769,0.0000,0.4487,2.65,0.65,0.0000,78
3,01,03,0.3971,0.1176,0.0000,0.4853,3.16,0.98,0.0000,68
4,01,04,0.3203,0.1046,0.0000,0.5686,3.92,1.18,0.0000,153
...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.2753,0.0531,0.0173,0.6432,3.18,0.65,0.0049,810
668,30,20,0.2423,0.0543,0.0220,0.6725,3.18,0.67,0.0059,681
669,30,21,0.2374,0.0747,0.0154,0.6549,3.33,0.61,0.0154,455
670,30,22,0.2667,0.0578,0.0000,0.6667,3.16,0.57,0.0000,450


In [24]:
#Create a column for hour, minute and second in df_captain
df_captain['hour'] = df_captain['time'].apply(lambda x: x[0:2])
df_captain['minute'] = df_captain['time'].apply(lambda x: x[2:4])
df_captain['second'] = df_captain['time'].apply(lambda x: x[4:])
df_captain['date'] = df_captain['yyyymmdd'].apply(lambda x: x[6:])

#Create a column for getting the 20_sec_batch
df_captain['minute'] = df_captain['minute'].astype(float)
df_captain['second'] = df_captain['second'].astype(float)
df_captain['20_sec_batch'] = pd.cut(df_captain['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df_captain['count'] = 1
df_captain.head()

,pickup_location_hex_8,yyyymmdd,order_type,time,gross_captains,hour,minute,second,date,20_sec_batch,count
0,8860a25953fffff,20221011,app,075336,3,07,53.0,36.0,11,20-40,1
1,8860a2590bfffff,20220929,app,212939,4,21,29.0,39.0,29,20-40,1
2,8860a25931fffff,20221010,app,074228,2,07,42.0,28.0,10,20-40,1
3,8860a25901fffff,20221006,app,232858,7,23,28.0,58.0,06,40-60,1
4,8860a2593bfffff,20221007,app,221324,3,22,13.0,24.0,07,20-40,1


In [25]:
#Get the number of captains coming in each hour and every 20_sec_batch
captain_hourly = df_captain.groupby(['date', 'hour'])['gross_captains'].sum().reset_index()
captain_hourly = captain_hourly.rename(columns={'gross_captains': 'hourly_gross_captains'})

captain_20_sec_batch = df_captain.groupby(['date', 'hour', 'minute', '20_sec_batch'])['gross_captains'].sum().reset_index()

#Get the average number of captains available in each 20_sec_batch per hour
captain_20_sec_average = captain_20_sec_batch.groupby(['date', 'hour'])['gross_captains'].mean().reset_index()
captain_20_sec_average['gross_captains'] = captain_20_sec_average['gross_captains'].apply(lambda x: round(x, 2))
captain_20_sec_average = captain_20_sec_average.rename(columns= {'gross_captains':'20_sec_gross_captains'})

#Merge the hourly captains and 20_sec_captains with the metrics table
metrics_df = metrics_df.merge(captain_hourly, on = ['date', 'hour'], how = 'left')
metrics_df = metrics_df.merge(captain_20_sec_average, on = ['date', 'hour'], how = 'left')
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains
0,01,00,0.3889,0.0617,0.0000,0.5494,2.96,0.84,0.0000,162,443,2.46
1,01,01,0.4545,0.0505,0.0000,0.4949,2.81,0.71,0.0000,99,267,1.48
2,01,02,0.4744,0.0769,0.0000,0.4487,2.65,0.65,0.0000,78,206,1.14
3,01,03,0.3971,0.1176,0.0000,0.4853,3.16,0.98,0.0000,68,207,1.15
4,01,04,0.3203,0.1046,0.0000,0.5686,3.92,1.18,0.0000,153,530,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.2753,0.0531,0.0173,0.6432,3.18,0.65,0.0049,810,2025,11.25
668,30,20,0.2423,0.0543,0.0220,0.6725,3.18,0.67,0.0059,681,1750,9.72
669,30,21,0.2374,0.0747,0.0154,0.6549,3.33,0.61,0.0154,455,1132,6.29
670,30,22,0.2667,0.0578,0.0000,0.6667,3.16,0.57,0.0000,450,1106,6.14


In [27]:
#Load the deamnd_summary and add it to the metrics
df_demand = pd.read_csv('demand_hyderabad.csv')
df_demand = df_demand.sort_values(by = ['date','hour'], ascending = True).reset_index(drop = True)
metrics_df['rides_per_hour'] = df_demand['rides_per_hour']
metrics_df['rides_per_20_sec'] = df_demand['rides_per_20_sec']
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec
0,01,00,0.3889,0.0617,0.0000,0.5494,2.96,0.84,0.0000,162,443,2.46,54.0,0.30
1,01,01,0.4545,0.0505,0.0000,0.4949,2.81,0.71,0.0000,99,267,1.48,33.0,0.18
2,01,02,0.4744,0.0769,0.0000,0.4487,2.65,0.65,0.0000,78,206,1.14,26.0,0.14
3,01,03,0.3971,0.1176,0.0000,0.4853,3.16,0.98,0.0000,68,207,1.15,23.0,0.13
4,01,04,0.3203,0.1046,0.0000,0.5686,3.92,1.18,0.0000,153,530,2.94,51.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.2753,0.0531,0.0173,0.6432,3.18,0.65,0.0049,810,2025,11.25,270.0,1.50
668,30,20,0.2423,0.0543,0.0220,0.6725,3.18,0.67,0.0059,681,1750,9.72,227.0,1.26
669,30,21,0.2374,0.0747,0.0154,0.6549,3.33,0.61,0.0154,455,1132,6.29,152.0,0.84
670,30,22,0.2667,0.0578,0.0000,0.6667,3.16,0.57,0.0000,450,1106,6.14,150.0,0.83


In [28]:
#Load the decrease in FM due to batching and add it to the metrics
decrease_FM = pd.read_excel('decrease_FM_batching.xlsx')
decrease_FM = decrease_FM.sort_values(by = ['date', 'hour'], ascending = True).reset_index(drop = True)
metrics_df['FM_decrease'] = decrease_FM['FM_decrease']
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec,FM_decrease
0,01,00,0.3889,0.0617,0.0000,0.5494,2.96,0.84,0.0000,162,443,2.46,54.0,0.30,-0.0767
1,01,01,0.4545,0.0505,0.0000,0.4949,2.81,0.71,0.0000,99,267,1.48,33.0,0.18,-0.0470
2,01,02,0.4744,0.0769,0.0000,0.4487,2.65,0.65,0.0000,78,206,1.14,26.0,0.14,0.0378
3,01,03,0.3971,0.1176,0.0000,0.4853,3.16,0.98,0.0000,68,207,1.15,23.0,0.13,-0.0172
4,01,04,0.3203,0.1046,0.0000,0.5686,3.92,1.18,0.0000,153,530,2.94,51.0,0.28,-0.0111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.2753,0.0531,0.0173,0.6432,3.18,0.65,0.0049,810,2025,11.25,270.0,1.50,-0.0421
668,30,20,0.2423,0.0543,0.0220,0.6725,3.18,0.67,0.0059,681,1750,9.72,227.0,1.26,-0.0628
669,30,21,0.2374,0.0747,0.0154,0.6549,3.33,0.61,0.0154,455,1132,6.29,152.0,0.84,-0.0342
670,30,22,0.2667,0.0578,0.0000,0.6667,3.16,0.57,0.0000,450,1106,6.14,150.0,0.83,-0.0669


In [29]:
metrics_df.to_csv('metrics_hyderabad.csv', index = False)